# Scikit_learn 라이브러리 활용

## 1. XOR 연산을 통한 간단한 학습

In [2]:
from sklearn import svm

### 1-1) xor 데이터 입력

In [5]:
##1)입력값 [명제1,명제2,result]


xor_data =  [
    [0,0,0],
    [1,0,1],
    [0,1,1],
    [1,1,0]
]


### 1-2) xor 데이터를 종속과 독립으로 나누기

In [10]:
##2)xor은 기본적으로 지도학습 이다.
    # 그럼으로 x y를 나누어서 학습시켜야한다.
    # x = [두 명제 쌍]
    # y = [참거짓]

#### 데이터를 나눌때 쓰는 기법이다 및에 쓰는 기법 써먹어보자
data = []
label = []
for index in xor_data:
    p = index[0]
    q = index[1]
    r = index[2]
    data.append([p,q])
    label.append(r)
print("data :",data,"\n","label :",label)

data : [[0, 0], [1, 0], [0, 1], [1, 1]] 
 label : [0, 1, 1, 0]


### 1-3) 데이터 학습시키기

In [12]:
clf = svm.SVC()
### fit 함수의 매개변수는 "fit(x값,y값)"형태로 한다
clf.fit(data,label)
clf

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### 1-4) 데이터 예측하기

In [15]:
### 여기서 예측값이란, data값( 즉 x 값을)주고 앞에서 학습한 데이터를 가지고 알고리즘을 돌려보는 것이다.

pre = clf.predict(data)
print("예측결과 :",pre)

예측결과 : [0 1 1 0]


### 1-5) 학습 알고리즘 검증하기

In [20]:
#### 위에서 예측한 결과가 앞에서 설정한 x,y 쌍과 일치하는지 확인하는 절차이다.
### 검증 함수는 다음과 같이
    ## "앞의 쌍과 일치하는 횟수 / 전체 반복 횟수 " 를 통해서 정답률을 측정한다.

true = 0; total_action = 0
## 한줄에 변수 2개를 설정하는 방법!!!
    ## "var_1 = []; var_2 = []"
    
for idx,answer in enumerate(label):
    if answer == pre[idx]:
        true += 1
    total_action += 1

print("정답횟수 :",true,"\n시행횟수 :",total_action,"\n정답률 :",true/total_action)


정답횟수 : 4 
시행횟수 : 4 
정답률 : 1.0


#### +) etc

In [21]:
pre_2 = clf.predict([1,1])
pre_2

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([0])

### 1-6) 검증하는 함수를 이용해서해보자

In [2]:
import pandas as pd
from sklearn import svm,metrics

In [39]:
#### sklearn에 metrics 함수를 통해서 결과값 pre 와 기존의 label 값을 일치 확률을 구한다.
    ## 그러기 위해서는 데이터를 "data_Frame" 형태로 변환 해야 한다.

xor_input = [
    [0,0,0],
    [1,0,1],
    [0,1,1],
    [1,1,0]
]

xor_df = pd.DataFrame(xor_input)
xor_df


,0,1,2
0,0,0,0
1,1,0,1
2,0,1,1
3,1,1,0


In [44]:
xor_data = xor_df.ix[:,0:1]
xor_label = xor_df.ix[:,2]

clf_2 = svm.SVC()
clf_2.fit(xor_data,xor_label)
pre_2 = clf_2.predict(xor_data)

#### 여기서 부터가 핵심 자동으로 일치를 확인해 주는 함수를 이용!!!!

ac_score = metrics.accuracy_score(pre,xor_label)
#ac_score = metrics.accuracy_score(xor_label,pre)
#### 위의 pre 와 xor_label 의 순서를 뒤바꾸어서 입력해도 동일한 결과가 나온다!!!
    ## 심지어 입력값을 잘못 주어서 당연히 한 값은 1보다 커야 하지만 그래도 동일한 결과가 나온다. 
    ## 이는 이 함수는 어떤것이 정답지이고, 어떤것이 시험 대상인지 안다는 것이다.
print("정답률 :",ac_score)

정답률 : 1.0


## 2. 150개의 row 로 이루어진 붓꽃 품종 분류하기!

In [51]:
#### 한번에 여러개의 라이브러리 불러오는 방법 = ","을 통해서 구분자를 준다!

from sklearn import svm,metrics
import random,re

### 2-1) csv 데이터 읽어오기( 한줄씩 읽어오기)

    자료 출처 : https://github.com/pydata/pandas/blob/master/pandas/tests/data/iris.csv
    
    

In [58]:
csv = []

with open('iris.csv','r',encoding= 'utf-8') as fp:
    for line in fp:
        line = line.strip()## 줄바꿈 제거하기!
        cols = line.split(',') ## ","을 기준으로 자르기( 구분하기 !)
        ### 만약에 숫자라면 임의의 함수 ( 숫자를 실수로 변화 시켜주는 함수)를 통해서 실수로 변화 시켜라 숫자가 아니면 그냥 진행
        ## 그결과를 새로운 변수 fn 에 저장해라
        print("Debuggin_split(',') :",cols)
        fn = lambda n : float(n) if re.match(r'[0-9\.]+$',n) else n
        print("Debugging_lambda :",fn)
        ## cols의 값들을 fn에 넣어서 cols를 변경시켜라
        cols = list(map(fn,cols))
        print("Debugging_map :",cols,"\n=========================================")
        ### 보면 위의 cols의 숫자는 문자형태이다. 그러나 및의 cols의 숫자는 int 형태이다
        csv.append(cols)
        

Debuggin_split(',') : ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Name']
Debugging_lambda : <function <lambda> at 0x0000025FF5123620>
Debugging_map : ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Name'] 
Debuggin_split(',') : ['5.1', '3.5', '1.4', '0.2', 'Iris-setosa']
Debugging_lambda : <function <lambda> at 0x0000025FF5135510>
Debugging_map : [5.1, 3.5, 1.4, 0.2, 'Iris-setosa'] 
Debuggin_split(',') : ['4.9', '3.0', '1.4', '0.2', 'Iris-setosa']
Debugging_lambda : <function <lambda> at 0x0000025FF5135EA0>
Debugging_map : [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'] 
Debuggin_split(',') : ['4.7', '3.2', '1.3', '0.2', 'Iris-setosa']
Debugging_lambda : <function <lambda> at 0x0000025FF4CAE048>
Debugging_map : [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'] 
Debuggin_split(',') : ['4.6', '3.1', '1.5', '0.2', 'Iris-setosa']
Debugging_lambda : <function <lambda> at 0x0000025FF4CAEB70>
Debugging_map : [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'] 
Debuggin_split(',') : ['5.0', '3.6', '1.4', '0

#### +a) lambda 와 map 함수에 관해서
    
    lambda 함수 형태 = "var = lambda var_2 : 참일때 함수 if 참일 조건 else 거짓일 조건"
        ex) fn = lambda n : n * 2 if n== 1 else n
        만약 n이 1이면 2를 곱하고 그외는 그냥 내비두어라.
        
        + 위에 fn 은 함수이다. 여기서 나온 값들을 리스트와 시키는 것은 map 함수를 쓴다.
        
        
    map 함수 형태 : list(map(함수,함수의 정의역이 있는곳= 값들을 저장하고 싶은 곳))

In [1]:
## 읽어오는 부분 다시 정리

csv_2 = []
with open('iris.csv','r',encoding = 'utf-8') as fp:
    for line in fp:
        line = line.strip()
        cols = line.split('n')
        fn = lambda n : float(n) if re.match(r'^[0-9\.]+$',n) else n
        cols = list(map(fn,cols))
        csv_2.append(cols)

##### 위에 있는 칼럼 이름 (길이 폭 이름 써져 있는...) 지우기
del csv_2[0]
len(csv_2)


NameError: name 're' is not defined

### 2-2) 정렬되어 있는 데이터를 섞어버리기!
    from random.shuffle()

In [64]:
import random

###### 섞어버리자
random.shuffle(csv_2)

### 2-3) 학습전용 데이터와 테스트 전용 데이터 분활 하기! 

In [72]:
## 일단 전체 길이
total_len = len(csv_2)
## 학습데이터 = "train" 이라고 부른다. 테스트 데이터 = "test"
train_len = int(total_len * 2/3)
#### text_len은 그외이니까 굳이 설정 x

train_data = []
train_label = []
text_data = []
text_label = []

for row_num in range(total_len):
    data = csv_2[row_num][0:4]
    print(csv_2[148][3])
#    label = csv_2[row_num][4]
#    if row_num <= train_len:
#        train_data.append(data)
#        train_label.append(label)
#    else:
#        text_data.append(data)
#        text_label.append(label)

        

IndexError: list index out of range

In [3]:
import pandas as pd
from sklearn import svm,metrics
import random,re

In [66]:
### 오류나서 파일 읽고 train , text 나누는 것 까지 다시하기
csv_3 = []

with open("iris.csv","r",encoding = "utf-8") as fp:
    for line in fp:
        line = line.strip()# 벗기다 strip
        cols = line.split(",")
        fn = lambda n : float(n) if re.match(r'^[0-9\.]+$',n) else n
        cols = list(map(fn,cols))
        csv_3.append(cols)
        

del csv_3[0]

random.shuffle(csv_3)

total_len = len(csv_3)
train_len = int(total_len * 2/3)

train_data = []
train_label = []
text_data =[]
text_label = []

for row_num in range(total_len): ### range(total_len) = 0부터 149 까지
    data = csv_3[row_num][0:4]
    label = csv_3[row_num][4]
    if row_num < train_len:
        train_data.append(data)
        train_label.append(label)
    else:
        text_data.append(data)
        text_label.append(label)
        
        
### 이제 학습 및 정답률 확인
clf = svm.SVC()
clf.fit(train_data,train_label)
pre = clf.predict(text_data)

score = metrics.accuracy_score(text_label,pre)
print("정답률 :",score)


####### 위에 random.shuffle 함수 때문에 정답률이 돌릴때마다 다르게 나온다.!!
### 그러나 중요한 것은 정답률이 높다는 것이다!

정답률 : 0.98


##### +a) 햇갈리는거 정리
    list[n:m] = n+1 ~ m 번쨰 까지
    list[m] = m+1 번째 
    range(k) = 0~(k-1) 까지의 수
             = k 개의 숫자 나열 이다!( 0 ~k-1 ) 까지니까 k개!
            

## 3. 붓꽃데이터를 이용한, 자동으로 test와 train data 를 나누어 주는 툴이용!

#### with) from sklearn.model_selection import train_test_split

In [1]:
import pandas as pd
from sklearn import svm,metrics
from sklearn.model_selection import train_test_split

In [11]:
csv_4 = pd.read_csv("iris.csv")

csv_4_colnames = list(csv_4.columns.values)
csv_4_colnames

['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Name']

In [38]:
data_4 = csv_4[['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']]
label_4 = csv_4['Name']

### 위에 함수 응용해서 한번에 할 수도 있다.
#data_4 = csv_4[list(csv_4.columns.values)[0:4]]
#label_4 = csv_4[list(csv_4.columns.values)[4]]

train_data, test_data, train_label, test_label = \
    train_test_split(data_4,label_4)
    
     #### \이후에 무조건  다음칸으로 넘기고 tap을 눌러야 한다
### 당연히 train_test_split()함수는 돌릴떄마다 결과가 렌덤하게 달라진다.

In [39]:
clf_4 = svm.SVC()
clf_4.fit(train_data,train_label)
pre_4 = clf_4.predict(test_data)

as_score_4 = metrics.accuracy_score(test_label,pre_4)
as_score_4

0.97368421052631582